### Task 1: Loading Transactions from a CSV File

In [25]:
import csv
from datetime import datetime

def add_transactions(filename = 'financial_transactions_short.csv'):
    transactions = []
    try:
        with open(filename, 'r', newline='') as file:
            csv_reader = csv.DictReader(file)
            for row in csv_reader:
                transactions.append(row)                
    except FileNotFoundError:
        print(f"Error: The file '{filename}' was not found.")
        return None
    transaction_count = len(transactions)

    print("--- Add New Transaction ---")
    
    transaction_id = transaction_count + 1
    transaction_date = datetime.now()
    date = transaction_date.strftime("%m/%d/%Y")
    customer_id = input("Please enter your numeric customer id: ")
    type = input("Enter transaction type (e.g. - credit or debit): ").strip()
    
    if not type:
        print("Transaction type cannot be empty. Aborting...")
        return
    
    while True:
        amount_str = input("Enter amount: ").strip()
        try:
            amount = float(amount_str)
            break
        except ValueError:
            print("Invalid amount. Please enter numerical value.")
    
    description = input("Enter description for transaction: ").strip()


    new_row = {'transaction_id': transaction_id, 'date': date, 'customer_id': customer_id, 'amount': amount, 'type': type, 'description': description}

    try:
        fieldnames = ['transaction_id', 'date', 'customer_id', 'amount', 'type', 'description']
        with open(filename, 'a', newline='') as file:
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            if file.tell() == 0:
                writer.writeheader()
            writer.writerow(new_row)
        print(f"Transaction {transaction_id} added successfully!")
        print(f"Date: {date}")
        print(f"Customer ID: {customer_id}")
        print(f"Amount: {amount}")
        print(f"Transaction Type: {type}")
        print(f"Description: {description}")
    except IOError as e:
        print(f"Error writing to file '{filename}': {e}")

def load_transactions(filename = 'financial_transactions_short.csv'):
    ERROR_LOG_FILE = 'errors.txt'
    try:
        with open(ERROR_LOG_FILE, 'w') as f:
            f.write(f"[{datetime.now()}] --- Transaction Processing Log Started ---\n")
        print(f"Log file '{ERROR_LOG_FILE}' has been created/cleared.")
    except IOError as e:
        print(f"Failed to initialize error log file '{ERROR_LOG_FILE}': {e}")
    
    transactions = []
    
    try:
        with open(filename, 'r', newline='') as file:
            csv_reader = csv.DictReader(file)
            for row in csv_reader:
                transactions.append(row)                
    except FileNotFoundError:
        print(f"Error: The file '{filename}' was not found.")
        return None
    
    for item in transactions:
        date_str = item['date']
        try:
            parsed_date = datetime.strptime(date_str, "%Y-%m-%d")
        except ValueError:
            try:
                parsed_date = datetime.strptime(date_str, "%d-%m-%Y")
            except ValueError:
                try:
                    parsed_date = datetime.strptime(date_str, "%m/%d/%Y")
                except ValueError:
                    parsed_date = None

        if parsed_date:
            item['date'] = parsed_date.strftime("%Y, %M, %d")
        else:
            with open(ERROR_LOG_FILE, 'a') as f:
                f.write(f"Error: Skipping transaction {item['transaction_id']}. Invalid date.\n")

    for item in transactions:
        try:
            if not item['amount'] or item['amount'].strip() =="":
                new_amount = 0.0
                print(f"Warning: Empty amount found for transaction {item['transaction_id']}. Setting to 0.0")
            else:
                new_amount = float(item['amount'])
            
            if item['type'] == "debit":
                item['amount'] = new_amount * -1
            elif item['type'] == "credit":
                item['amount'] = new_amount            
        except ValueError:
            with open(ERROR_LOG_FILE, 'a') as f:
                f.write(f"Error: Could not convert amount to float in transaction {item['transaction_id']}. Invalid amount.")



    print(f"There are {len(transactions)} transactions.")
    print(f"Transaction processing complete. Check {ERROR_LOG_FILE} for any logged issues.")

    return transactions

       
add_transactions()


--- Add New Transaction ---
Transaction 21 added successfully!
Date: 05/24/2025
Customer ID: 542
Amount: 165.03
Transaction Type: credit
Description: test transaction


### Task 2: Adding and Viewing Transactions

In [ ]:
# User input
# Input validation
# Functions
# Looping through lists
# Format output
# Debug interactions
# Reject invalid transaction types in add_transaction
# Add an option to view_transactions to filter by type
# Suggest customer IDs from existing transactions
# Format dates as "Oct 26, 2020" in view_transactions

def add_transaction(transactions):

def view_transactions(transactions):
    

### Task 3: Updating and Deleting Transactions

In [ ]:
# Looping through lists
# Functions
# List operations
# Conditionals
# Debug lists
# Prevent invalid type updates in update_transaction
# Show transaction details before deletion
# Allow updating multiple fields at once
# Add a cancel option for updates/deletions

def update_transaction(transactions):


def delete_transaction(transactions):

### Analyzing Financial Data

In [ ]:
# Looping
# Dictionaries
# Calculations
# Formatting
# Algorithms
# Show the customer with the highest debit amount
# Calculate percentage of total amount by type
# Analyze transactions from 2022 only
# Save analysis to analysis.txt

def analyze_finances(transactions):

### Task 5: Saving Transactions and Generating Reports

In [ ]:
# Writing CSV files
# File error handling
# Formatting
# Debug file output
# Add a timestamp to the file report name
# Back up the original CSV before saving
# Include transaction date range in report
# Handle file-writing errors

def save_transactions(transactions, filename='financial_transactions.csv'):

def generate_report(transactions, filename='report.txt'):